<a href="https://colab.research.google.com/github/hdansu/DataScience/blob/master/Airline_travel_corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## install packages
!pip install beautifulsoup4
!pip install Requests

Next few cells to illustrate 
---

In [0]:
import requests

url = 'https://www.tsa.gov/coronavirus/passenger-throughput'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
html = response.content

In [0]:
import requests
from bs4 import BeautifulSoup
#import BeautifulSoup

url = 'https://www.tsa.gov/coronavirus/passenger-throughput'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
html = response.content

soup = BeautifulSoup(html)
print (soup.prettify())

**Setup script to scrape info from TSA site**
---


In [0]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.tsa.gov/coronavirus/passenger-throughput'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
html = response.content

soup = BeautifulSoup(html)

In [0]:
table = soup.find('table')

list_of_rows = []
for row in table.findAll('tr'):
    list_of_cells = []
    for cell in row.findAll('td'):
        text = cell.text.replace('&nbsp;', '')
        list_of_cells.append(text)
    list_of_rows.append(list_of_cells)

## Reference: https://first-web-scraper.readthedocs.io/en/latest/

**Save data into table**
---

In [7]:
#load packages

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [8]:
df = pd.DataFrame(data=list_of_rows)
df.head()

,0,1,2
0,Date,Total Traveler Throughput,Total Traveler Throughput(1 Year Ago - Same We...
1,4/22/2020,"98,968","2,254,209"
2,4/21/2020,"92,859","2,227,475"
3,4/20/2020,"99,344","2,594,171"
4,4/19/2020,"105,382","2,356,802"


In [0]:
df.columns = df.iloc[0] #set the header row as the df header
df = df[1:]             #take the data less the header row (drop header row)

# Reformat date column
df['Date'] = pd.to_datetime(df.Date)
# df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')

In [10]:
df.head()

,Date,Total Traveler Throughput,Total Traveler Throughput(1 Year Ago - Same Weekday)
1,2020-04-22,"98,968","2,254,209"
2,2020-04-21,"92,859","2,227,475"
3,2020-04-20,"99,344","2,594,171"
4,2020-04-19,"105,382","2,356,802"
5,2020-04-18,"97,236","1,988,205"


In [11]:
# Rename columns
df = df.rename(columns={'Total Traveler Throughput': 'Current Year', 'Total Traveler Throughput(1 Year Ago - Same Weekday)': 'Prior Year (same weekday)'})

df.head()

,Date,Current Year,Prior Year (same weekday)
1,2020-04-22,"98,968","2,254,209"
2,2020-04-21,"92,859","2,227,475"
3,2020-04-20,"99,344","2,594,171"
4,2020-04-19,"105,382","2,356,802"
5,2020-04-18,"97,236","1,988,205"


**Plot data**
---

v1.0

In [12]:
# Using plotly express
import plotly.express as px

fig = px.line(df, x='Date', y='Current Year', title='Total Traveler Throughput')
fig.show()

v2.0

In [22]:
## Using simple Plotly graphs

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Current Year'],
                    mode='lines',
                    name='Current Year'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Prior Year (same weekday)'],
                    mode='lines',
                    name='Prior Year'))

fig.update_layout(title='Total Traveler Throughput')

fig.update_layout(xaxis_type='date')

fig.show()

v2.1

In [13]:
## Using simple Plotly graphs

import plotly.graph_objects as go

fig = go.Figure()

#Reference: https://plotly.com/python/line-charts/

# draw lines
fig.add_trace(go.Scatter(x=df['Date'], y=df['Current Year'],
                    mode='lines',
                    name='Current Year',
                    line=dict(color='rgb(49,130,189)', width=3),
                    ))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Prior Year (same weekday)'],
                    mode='lines',
                    name='Prior Year',
                    line=dict(color='rgb(115,115,115)', width=2),
                    ))

# set up plot area

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)

# add annotations
annotations = []

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Total Traveler Throughput',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='https://www.tsa.gov/coronavirus/passenger-throughput',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)
#fig.update_layout(title='Total Traveler Throughput')
#fig.update_layout(xaxis_type='date')

fig.show()

In [0]:
x_data = df['Date']
y1_data = df['Current Year']
y2_data = df['Prior Year (same weekday)']

x_data[0]

In [68]:
## Using simple Plotly graphs
#Reference: https://plotly.com/python/line-charts/

import plotly.graph_objects as go

fig = go.Figure()

x_data = df['Date']
y1_data = df['Current Year']
y2_data = df['Prior Year (same weekday)']

# Draw lines
fig.add_trace(go.Scatter(x=x_data, y=y1_data,
                    mode='lines',
                    name='Current Year',
                    line=dict(color='rgb(49,130,189)', width=3),
                    ))
fig.add_trace(go.Scatter(x=x_data, y=y2_data,
                    mode='lines',
                    name='Prior Year',
                    line=dict(color='rgb(115,115,115)', width=2),
                    ))

# Set up plot area
fig.update_layout(
    xaxis=dict(
        zeroline=False,
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        zerolinecolor='white',
        showline=False,
        showticklabels=True,
        gridcolor='rgb(204, 204, 204)',
        gridwidth=1,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        tickfont=dict(
          family='Arial',
          size=12,
          color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

# add annotations
annotations = []

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Total Traveler Throughput',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.4, y=-0.2,
                              xanchor='right', yanchor='top',
                              text='Source: Department of Homeland Security',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)
#fig.update_layout(title='Total Traveler Throughput')
#fig.update_layout(xaxis_type='date')

fig.show()

- tweak axis spacing
- correct zero line
- add subtitle
- tweak fonts
- horizontal dashes for historical average
- add highlight(vertical shading) for last week
- add watermark / image